In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
#alllines-reduced.txt includes Act I, Scenes I-III of Henry IV
with open('datasets/alllines-reduced.txt', 'r') as file:
    text = file.read()
    lines = text.lower().split('\n')

#Define words, vocabulary size and sequences of words as lines
#from keras.preprocessing.text import text_to_word_sequence, Tokenizer
words = tf.keras.preprocessing.text.text_to_word_sequence(text)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(words)
vocabulary_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(lines)

#Find subsequences 
subsequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        subsequence = sequence[:i+1]
        subsequences.append(subsequence)

In [3]:
vocabulary_size

1464

In [4]:
#from keras.preprocessing.sequence import pad_sequences
sequence_length = max([len(sequence) for sequence in sequences])
sequences = tf.keras.preprocessing.sequence.pad_sequences(subsequences, maxlen=sequence_length, padding='pre')

In [5]:
#from keras.utils import to_categorical
x, y = sequences[:,:-1],sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=vocabulary_size)

In [6]:
print(x.shape,y.shape)

(4339, 15) (4339, 1464)


In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

In [8]:
model = tf.keras.models.Sequential()

In [9]:
#1. An embedding layer with the following parameters:
    #The input dimension is vocabulary_size
    #The output dimension is 100
    #The input length is sequence_length - 1
model.add(tf.keras.layers.Embedding(input_dim=vocabulary_size, output_dim=100, input_length=sequence_length-1))

#2. An LSTM layer with 100 units
model.add(tf.keras.layers.LSTM(100))

#3. A dropout layer with a dropout rate of 10%
model.add(tf.keras.layers.Dropout(0.10))

#4.A dense layer with the following parameters:
    #Activation function is softmax
    #The number of units is vocabulary_size
model.add(tf.keras.layers.Dense(units=vocabulary_size, activation='softmax'))

In [10]:
#Build the network using the following parameters:
    #Optimizer: Adam
    #Loss function: categorical_crossentropy
    #Metrics: accuracy
    #Epochs: 500
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 100)           146400    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1464)              147864    
Total params: 374,664
Trainable params: 374,664
Non-trainable params: 0
_________________________________________________________________


In [12]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [13]:
model.fit(x_train, y_train, epochs=500, batch_size=32,callbacks=[callback])

Epoch 1/500
109/109 [==============================] - 1s 12ms/step - loss: 6.7241 - accuracy: 0.0314
Epoch 2/500
109/109 [==============================] - 1s 11ms/step - loss: 6.1954 - accuracy: 0.0409
Epoch 3/500
109/109 [==============================] - 1s 11ms/step - loss: 6.0587 - accuracy: 0.0380
Epoch 4/500
109/109 [==============================] - 1s 12ms/step - loss: 5.9676 - accuracy: 0.0464
Epoch 5/500
109/109 [==============================] - 1s 11ms/step - loss: 5.8812 - accuracy: 0.0550
Epoch 6/500
109/109 [==============================] - 1s 11ms/step - loss: 5.7950 - accuracy: 0.0573
Epoch 7/500
109/109 [==============================] - 1s 11ms/step - loss: 5.7169 - accuracy: 0.0666
Epoch 8/500
109/109 [==============================] - 1s 11ms/step - loss: 5.6425 - accuracy: 0.0691
Epoch 9/500
109/109 [==============================] - 1s 11ms/step - loss: 5.5713 - accuracy: 0.0726
Epoch 10/500
109/109 [==============================] - 1s 11ms/step - loss: 5.500

109/109 [==============================] - 1s 13ms/step - loss: 0.7675 - accuracy: 0.8277
Epoch 81/500
109/109 [==============================] - 1s 11ms/step - loss: 0.7504 - accuracy: 0.8369
Epoch 82/500
109/109 [==============================] - 1s 11ms/step - loss: 0.7276 - accuracy: 0.8413
Epoch 83/500
109/109 [==============================] - 1s 11ms/step - loss: 0.7123 - accuracy: 0.8441
Epoch 84/500
109/109 [==============================] - 1s 11ms/step - loss: 0.6948 - accuracy: 0.8476
Epoch 85/500
109/109 [==============================] - 1s 11ms/step - loss: 0.6745 - accuracy: 0.8485
Epoch 86/500
109/109 [==============================] - 1s 11ms/step - loss: 0.6596 - accuracy: 0.8496
Epoch 87/500
109/109 [==============================] - 1s 11ms/step - loss: 0.6591 - accuracy: 0.8545
Epoch 88/500
109/109 [==============================] - 1s 11ms/step - loss: 0.6435 - accuracy: 0.8522
Epoch 89/500
109/109 [==============================] - 1s 11ms/step - loss: 0.6303 - 

In [14]:
loss, accuracy = model.evaluate(x_test, y_test)

28/28 [==============================] - 0s 4ms/step - loss: 12.6254 - accuracy: 0.0553


In [15]:
print("loss:",loss)
print("accuracy:",accuracy)

loss: 12.625411033630371
accuracy: 0.0552995391190052
